---
title: "Elite Dangerous: Guardians I"
author: "CMDR immerlicht"
date: "3310-02-01"
categories: [elite-dangerous, code, analysis]
image: "HighResScreenShot_2023-01-06_19-53-16_DxO.jpg"
echo: false
---



The Guardians we call them, a lost, enigmatic civilization that left us beacons and ruins, millions of years old, yet all still functional. They were technologically advanced and colonized part of the Orion arm long before humans learned to travel between stars. 

A few years after their initial discovery a remarkable 300 systems have been discovered with remains of this mysterious civilization.






[Canonn Research / The Guardians](https://canonn.science/codex/the-guardians/)



In [ ]:
#
import os
import numpy as np
import pandas as pd
from itertools import accumulate, permutations, combinations, product
from sklearn.cluster import DBSCAN

# Terzijde/data/guardian/Canonn - Guardians - Brain Tree Sites.csv
# Terzijde/terzijde/posts/ed-guardians-1/index.ipynb
guardiandata_path = os.path.join(os.getcwd(), '..', '..', '..', 'data', 'guardian')

In [ ]:
guardiandata_files = {n.split(' - ')[2].split('.')[-2]:os.path.join(guardiandata_path, n) for n in os.listdir(guardiandata_path) if 'Canonn - Guardians' in n}
guardiandata = {n:pd.read_csv(p) for n,p in guardiandata_files.items()}
soi = ['Ruins','Structures','Beacons']


inter_columns = list([i for i in accumulate([set(guardiandata['Guardian '+n].columns) for n in soi], lambda D1, D2: D1 & D2)][-1])
union_columns = list([i for i in accumulate([set(guardiandata['Guardian '+n].columns) for n in soi], lambda D1, D2: D1 | D2)][-1])

column_order_inter = {c:i for c,i in zip(guardiandata['Guardian Beacons'].columns, range(len(guardiandata['Guardian Beacons'].columns))) }
inter_columns = sorted(inter_columns, key=lambda I:column_order_inter.get(I,100))
union_columns = sorted(union_columns, key=lambda I:column_order_inter.get(I,100))

In [ ]:
import json

with open(f"guardian-beacons.json", 'wt') as of:
    json.dump(dict(markers=[
            dict(
                pin='red',
                text=str(row),
                **{c:v for c,v in zip(['x','y','z'], row[['x','y','z']])}
            )
            for index,row in guardiandata['Guardian Beacons'].iterrows()
        ]), of, indent=3)

In [ ]:
#[row for index,row in guardiandata['Guardian Beacons'].iterrows()]

In [ ]:
soi_systems = {
    r['System Name']:dict(
        coord=np.asarray([r[c] for c in ['x', 'y', 'z']]),
        info= np.asarray([g] + [r[c] for c in inter_columns])
    )
    for g in soi for i, r in guardiandata['Guardian '+g].iterrows()
}

soi_systemnames = np.vstack([v['info'] for k,v in soi_systems.items()])
soi_coordinates = np.vstack([v['coord'] for k,v in soi_systems.items()])
#print(soi_coordinates.shape, soi_systemnames.shape)

In [ ]:
soi_systemnames = np.asarray(
    [[g]+[r[c] for c in inter_columns] for g in soi for i, r in guardiandata['Guardian '+g].iterrows()] )
soi_coordinates = np.asarray([[r[c] for c in ['x', 'y', 'z']] for g in soi for i, r in guardiandata['Guardian '+g].iterrows()])
#print(soi_coordinates.shape, soi_systemnames.shape)

In [ ]:
soi_systems = {r['System Name']:np.asarray([r[c] for c in ['x', 'y', 'z']]) for g in soi for i, r in guardiandata['Guardian '+g].iterrows()}
soi_systemnames = np.asarray(
    [[g]+[r[c] for c in inter_columns] for g in soi for i, r in guardiandata['Guardian '+g].iterrows()] )
soi_coordinates = np.asarray([[r[c] for c in ['x', 'y', 'z']] for g in soi for i, r in guardiandata['Guardian '+g].iterrows()])
#print(soi_coordinates.shape, soi_systemnames.shape)

In [ ]:
coord_clusters = DBSCAN(eps=120, min_samples=3).fit(soi_coordinates)
#print(np.unique(coord_clusters.labels_))
#print(np.count_nonzero(np.less(coord_clusters.labels_,0)))
#print([(l, np.count_nonzero(np.equal(coord_clusters.labels_,l))) for l in np.unique(coord_clusters.labels_)])


In [ ]:
outliers = np.copy(soi_coordinates[np.less(coord_clusters.labels_,0)])
outlier_systemnames = np.copy(soi_systemnames[np.less(coord_clusters.labels_,0)])

pd.DataFrame([[-1]+n.tolist()+c.tolist() for n,c in zip(outlier_systemnames, outliers)])


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-1,Ruins,1,SYNUEFE XR-H D11-102,357.34375,-49.34375,-74.75,F (White) Star,1 B,Rocky body,797,MauserPT7,357.34375,-49.34375,-74.75000
1,-1,Ruins,361,TRAPEZIUM SECTOR YU-X C1-2,573.59375,-339.46875,-1167.65625,K (Yellow-Orange) Star,1 A,Rocky body,253,CodexNecro81,573.59375,-339.46875,-1167.65625
2,-1,Structures,186,TRAPEZIUM SECTOR YU-X C1-2,573.59375,-339.46875,-1167.65625,K (Yellow-Orange) Star,1 A,Rocky body,253,Namix,573.59375,-339.46875,-1167.65625
3,-1,Structures,200,WREGOE BU-Y B2-0,1077.375,400.5625,-993.375,M (Red dwarf) Star,1 C,Icy body,1216,Marina Oatmeal,1077.37500,400.56250,-993.37500
4,-1,Structures,201,WREGOE BU-Y B2-0,1077.375,400.5625,-993.375,M (Red dwarf) Star,1 C,Icy body,1216,Marina Oatmeal,1077.37500,400.56250,-993.37500


In [ ]:
import json

with open(f"guardian-clusters.json", 'wt') as of:
    json.dump(dict(markers=[
            dict(
                pin='red',
                text='\n'.join([f"{cn:20}: {v}" for cn, v in zip(['type'] + inter_columns,row[0])]),
                **{c:v for c,v in zip(['x','y','z'], row[1])}
            )
            for row in [(n,c) for n,c in zip(outlier_systemnames, outliers)]
        ] + [
            dict(
                pin='cyan',
                text=f'Center of cluster {row[0]} with {row[2]} guardian sites',
                **{c:v for c,v in zip(['x','y','z'], row[1])}
            )
            for row in [
                (l, np.round(np.mean(soi_coordinates[np.equal(coord_clusters.labels_,l)], axis=0),2).tolist(), np.count_nonzero(np.equal(coord_clusters.labels_,l)) ) 
                for l in np.unique(coord_clusters.labels_) 
                if not l < 0]
        ]), of, indent=3)